In [1]:
import os
import pandas as pd

In [2]:
nyc_master_df = pd.DataFrame()
nyc_station_ref = pd.read_csv('Data/NYC Data/MTA_Subway_Stations_20250124.csv')

In [3]:
nyc_station_ref.head()

,GTFS Stop ID,Station ID,Complex ID,Division,Line,Stop Name,Borough,CBD,Daytime Routes,Structure,GTFS Latitude,GTFS Longitude,North Direction Label,South Direction Label,ADA,ADA Northbound,ADA Southbound,ADA Notes,Georeference
0,R01,1,1,BMT,Astoria,Astoria-Ditmars Blvd,Q,False,N W,Elevated,40.775036,-73.912034,Last Stop,Manhattan,0,0,0,NaN,POINT (-73.912034 40.775036)
1,R03,2,2,BMT,Astoria,Astoria Blvd,Q,False,N W,Elevated,40.770258,-73.917843,Astoria,Manhattan,1,1,1,NaN,POINT (-73.917843 40.770258)
2,R04,3,3,BMT,Astoria,30 Av,Q,False,N W,Elevated,40.766779,-73.921479,Astoria,Manhattan,0,0,0,NaN,POINT (-73.921479 40.766779)
3,R05,4,4,BMT,Astoria,Broadway,Q,False,N W,Elevated,40.761820,-73.925508,Astoria,Manhattan,0,0,0,NaN,POINT (-73.925508 40.76182)
4,R06,5,5,BMT,Astoria,36 Av,Q,False,N W,Elevated,40.756804,-73.929575,Astoria,Manhattan,0,0,0,NaN,POINT (-73.929575 40.756804)


In [4]:
len(nyc_station_ref['Station ID'].unique())

493

In [5]:
nyc_station_ref['Complex ID'] = nyc_station_ref['Complex ID'].apply(lambda x: str(x))
nyc_station_ref['Station ID'] = nyc_station_ref['Station ID'].apply(lambda x: str(x))
nyc_station_ref = nyc_station_ref.drop_duplicates(subset = ['Complex ID'])

In [6]:
def process_nyc_data(nyc_df, how='monthly'):

    nyc_df_processed = nyc_df.groupby(['transit_timestamp', 'station_complex_id', 'station_complex', 'borough', 'latitude', 'longitude']).agg({'ridership': 'sum'}).reset_index()
    nyc_df_processed['timestamp'] = nyc_df_processed['transit_timestamp'].apply(lambda x: pd.Timestamp(x))
    nyc_df_processed['year'] = nyc_df_processed['timestamp'].apply(lambda x: x.year)
    nyc_df_processed['month'] = nyc_df_processed['timestamp'].apply(lambda x: x.month)
    nyc_df_processed['day'] = nyc_df_processed['timestamp'].apply(lambda x: x.day)
    nyc_df_processed['hour'] = nyc_df_processed['timestamp'].apply(lambda x: x.hour)

    if how == 'yearly':
        nyc_df_processed = nyc_df_processed.groupby(['year', 'station_complex_id', 'station_complex', 'borough', 'latitude', 'longitude']).agg({'ridership': 'sum'}).reset_index()
        nyc_df_processed['unique_time'] = nyc_df_processed['year']
    elif how == 'monthly':
        nyc_df_processed = nyc_df_processed.groupby(['year', 'month', 'station_complex_id', 'station_complex', 'borough', 'latitude', 'longitude']).agg({'ridership': 'sum'}).reset_index()
        nyc_df_processed['unique_time'] = nyc_df_processed.apply(lambda x: pd.Timestamp(year=x.year, month=x.month, day=1), axis=1)
    elif how == 'daily':
        nyc_df_processed = nyc_df_processed.groupby(['year', 'month', 'day', 'station_complex_id', 'station_complex', 'borough', 'latitude', 'longitude']).agg({'ridership': 'sum'}).reset_index()
        nyc_df_processed['unique_time'] = nyc_df_processed.apply(lambda x: pd.Timestamp(year=x.year, month=x.month, day=x.day), axis=1)
    elif how == 'hourly':
        nyc_df_processed = nyc_df_processed.groupby(['transit_timestamp', 'hour', 'station_complex_id', 'station_complex', 'borough', 'latitude', 'longitude']).agg({'ridership': 'sum'}).reset_index()
        nyc_df_processed['unique_time'] = nyc_df_processed['transit_timestamp']

    nyc_combined_df = pd.merge(nyc_df_processed, nyc_station_ref, how = 'left', left_on = ['station_complex_id'], right_on = ['Complex ID'])
    nyc_combined_df = nyc_combined_df[nyc_combined_df['GTFS Stop ID'].notna()]

    return nyc_combined_df
    

In [7]:
filepath = 'Data/NYC Data/Monthly Ridership/'

nyc_dfs = []
frequencies = ['yearly', 'monthly', 'daily']

for frequency in frequencies:
    print(frequency)
    nyc_dfs = []
    nyc_master_df = pd.DataFrame()
    for file in os.listdir(filepath):
        if file.startswith(f'NYC_Ridership') and file.endswith('.csv'):
            print(file)
            temp_df = pd.read_csv(os.path.join(filepath, file))
            temp_df = process_nyc_data(temp_df, how=frequency)
            nyc_dfs.append(temp_df)
            del temp_df
    nyc_master_df = pd.concat(nyc_dfs)
    nyc_master_df.to_csv(f'Data/Master Sets/NYC Master/Yearly Combined/NYC_Combined_{frequency}.csv')


yearly
NYC_Ridership_2020_10.csv
NYC_Ridership_2020_11.csv
NYC_Ridership_2020_12.csv
NYC_Ridership_2020_7.csv
NYC_Ridership_2020_8.csv
NYC_Ridership_2020_9.csv
NYC_Ridership_2021_1.csv
NYC_Ridership_2021_10.csv
NYC_Ridership_2021_11.csv
NYC_Ridership_2021_12.csv
NYC_Ridership_2021_2.csv
NYC_Ridership_2021_3.csv
NYC_Ridership_2021_4.csv
NYC_Ridership_2021_5.csv
NYC_Ridership_2021_6.csv
NYC_Ridership_2021_7.csv
NYC_Ridership_2021_8.csv
NYC_Ridership_2021_9.csv
NYC_Ridership_2022_1.csv
NYC_Ridership_2022_10.csv
NYC_Ridership_2022_11.csv
NYC_Ridership_2022_12.csv
NYC_Ridership_2022_2.csv
NYC_Ridership_2022_3.csv
NYC_Ridership_2022_4.csv
NYC_Ridership_2022_5.csv
NYC_Ridership_2022_6.csv
NYC_Ridership_2022_7.csv
NYC_Ridership_2022_8.csv
NYC_Ridership_2022_9.csv
NYC_Ridership_2023_1.csv
NYC_Ridership_2023_10.csv
NYC_Ridership_2023_11.csv
NYC_Ridership_2023_12.csv
NYC_Ridership_2023_2.csv
NYC_Ridership_2023_3.csv
NYC_Ridership_2023_4.csv


C:\Users\shygu\AppData\Local\Temp\ipykernel_21056\436671665.py:13: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  temp_df = pd.read_csv(os.path.join(filepath, file))


NYC_Ridership_2023_5.csv
NYC_Ridership_2023_6.csv
NYC_Ridership_2023_7.csv
NYC_Ridership_2023_8.csv
NYC_Ridership_2023_9.csv
NYC_Ridership_2024_1.csv
NYC_Ridership_2024_10.csv
NYC_Ridership_2024_11.csv
NYC_Ridership_2024_12.csv
NYC_Ridership_2024_2.csv
NYC_Ridership_2024_3.csv
NYC_Ridership_2024_4.csv
NYC_Ridership_2024_5.csv
NYC_Ridership_2024_6.csv
NYC_Ridership_2024_7.csv
NYC_Ridership_2024_8.csv
NYC_Ridership_2024_9.csv
monthly
NYC_Ridership_2020_10.csv
NYC_Ridership_2020_11.csv
NYC_Ridership_2020_12.csv
NYC_Ridership_2020_7.csv
NYC_Ridership_2020_8.csv
NYC_Ridership_2020_9.csv
NYC_Ridership_2021_1.csv
NYC_Ridership_2021_10.csv
NYC_Ridership_2021_11.csv
NYC_Ridership_2021_12.csv
NYC_Ridership_2021_2.csv
NYC_Ridership_2021_3.csv
NYC_Ridership_2021_4.csv
NYC_Ridership_2021_5.csv
NYC_Ridership_2021_6.csv
NYC_Ridership_2021_7.csv
NYC_Ridership_2021_8.csv
NYC_Ridership_2021_9.csv
NYC_Ridership_2022_1.csv
NYC_Ridership_2022_10.csv
NYC_Ridership_2022_11.csv
NYC_Ridership_2022_12.csv
NYC_R

C:\Users\shygu\AppData\Local\Temp\ipykernel_21056\436671665.py:13: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  temp_df = pd.read_csv(os.path.join(filepath, file))


NYC_Ridership_2023_5.csv
NYC_Ridership_2023_6.csv
NYC_Ridership_2023_7.csv
NYC_Ridership_2023_8.csv
NYC_Ridership_2023_9.csv
NYC_Ridership_2024_1.csv
NYC_Ridership_2024_10.csv
NYC_Ridership_2024_11.csv
NYC_Ridership_2024_12.csv
NYC_Ridership_2024_2.csv
NYC_Ridership_2024_3.csv
NYC_Ridership_2024_4.csv
NYC_Ridership_2024_5.csv
NYC_Ridership_2024_6.csv
NYC_Ridership_2024_7.csv
NYC_Ridership_2024_8.csv
NYC_Ridership_2024_9.csv
daily
NYC_Ridership_2020_10.csv
NYC_Ridership_2020_11.csv
NYC_Ridership_2020_12.csv
NYC_Ridership_2020_7.csv
NYC_Ridership_2020_8.csv
NYC_Ridership_2020_9.csv
NYC_Ridership_2021_1.csv
NYC_Ridership_2021_10.csv
NYC_Ridership_2021_11.csv
NYC_Ridership_2021_12.csv
NYC_Ridership_2021_2.csv
NYC_Ridership_2021_3.csv
NYC_Ridership_2021_4.csv
NYC_Ridership_2021_5.csv
NYC_Ridership_2021_6.csv
NYC_Ridership_2021_7.csv
NYC_Ridership_2021_8.csv
NYC_Ridership_2021_9.csv
NYC_Ridership_2022_1.csv
NYC_Ridership_2022_10.csv
NYC_Ridership_2022_11.csv
NYC_Ridership_2022_12.csv
NYC_Rid

C:\Users\shygu\AppData\Local\Temp\ipykernel_21056\436671665.py:13: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  temp_df = pd.read_csv(os.path.join(filepath, file))


NYC_Ridership_2023_5.csv
NYC_Ridership_2023_6.csv
NYC_Ridership_2023_7.csv
NYC_Ridership_2023_8.csv
NYC_Ridership_2023_9.csv
NYC_Ridership_2024_1.csv
NYC_Ridership_2024_10.csv
NYC_Ridership_2024_11.csv
NYC_Ridership_2024_12.csv
NYC_Ridership_2024_2.csv
NYC_Ridership_2024_3.csv
NYC_Ridership_2024_4.csv
NYC_Ridership_2024_5.csv
NYC_Ridership_2024_6.csv
NYC_Ridership_2024_7.csv
NYC_Ridership_2024_8.csv
NYC_Ridership_2024_9.csv


In [8]:
# # VERY DANGEROUS DO NOT RUN UNLESS WILLING TO WAIT 40-60 HOURS

# nyc_dfs = []
# years = [2020, 2021, 2022, 2023, 2024]

# for year in years:
#     nyc_dfs = []
#     for file in os.listdir(filepath):
#         if file.startswith(f'NYC_Ridership_{year}') and file.endswith('.csv'):
#             print(file)
#             temp_df = pd.read_csv(os.path.join(filepath, file))
#             temp_df = process_nyc_data(temp_df, 'hourly')
#             nyc_dfs.append(temp_df)
#             del temp_df

#     try:
#         nyc_master_df = pd.concat(nyc_dfs)
#         nyc_master_df.to_csv(f'Data/Master Sets/NYC Master/Yearly Combined/NYC_Combined_{year}_hourly.csv')
#     except:
#         continue